<a href="https://colab.research.google.com/github/RaunakRaj2081/Machine-Learning_part1/blob/main/2_9_10_Text_Data_Pre_Processing_Use_Case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing the Dependencies

In [3]:
import numpy as np
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#Data Pre-Processing

In [8]:
# Use Python's built-in file handling to inspect the problematic line
file_path = '/content/WELFake_Dataset.csv'
target_line_number = 8288 # Error is at row 8287, which is likely line 8288 including header

try:
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Print lines around the target line number (e.g., 5 lines before and 5 lines after)
    start_index = max(0, target_line_number - 6) # -1 for 0-indexing, -5 to show previous 5 lines
    end_index = min(len(lines), target_line_number + 5) # +5 to show next 5 lines

    print(f"Inspecting lines from {start_index + 1} to {end_index} (0-indexed: {start_index} to {end_index-1}):")
    print("-" * 30) # Separator

    for i in range(start_index, end_index):
        # Print the line number and the raw line content (strip just trailing newline for clarity)
        print(f"Line {i + 1}: {lines[i].rstrip()}")

    print("-" * 30) # Separator

except FileNotFoundError:
    print(f"Error: File not found at {file_path}")
except Exception as e:
    print(f"An error occurred while reading the file: {e}")

Inspecting lines from 8283 to 8293 (0-indexed: 8282 to 8292):
------------------------------
Line 8283: 
Line 8284: Armstrong claims that ""the police are now taking more assets than the criminals,"" but this isn't exactly right: The FBI also tracks property losses from larceny and theft, in addition to plain ol' burglary. If you add up all the property stolen in 2014, from burglary, theft, motor vehicle theft and other means, you arrive at roughly $12.3 billion, according to the FBI. That's more than double the federal asset forfeiture haul.
Line 8285: 
Line 8286: One other point: Those asset forfeiture deposit amounts are not necessarily the best indicator of a rise in the use of forfeiture. ""In a given year, one or two high-dollar cases may produce unusually large amounts of money — with a portion going back to victims — thereby telling a noisy story of year-to-year activity levels,"" the Institute for Justice explains. A big chunk of that 2014 deposit, for instance, was the $1.7 b

In [10]:
# load the data to a pandas dataframe
# Added engine='python' and on_bad_lines='skip' to handle potential parsing errors
# on_bad_lines='skip' will skip rows that cause parsing issues. Use with caution.
# If you are using pandas version < 1.4.0, use error_bad_lines=False instead of on_bad_lines='skip'
news_data = pd.read_csv('/content/WELFake_Dataset.csv', engine='python', on_bad_lines='skip')

# You can also use 'warn' to see which lines are skipped:
# news_data = pd.read_csv('/content/WELFake_Dataset.csv', engine='python', on_bad_lines='warn')

In [11]:
# first 5 rows of the dataset
news_data.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


0 --> Real News

1 --> Fake News

In [12]:
news_data.shape

(18979, 4)

In [13]:
# checking for missing values
news_data.isnull().sum()

,0
Unnamed: 0,0
title,121
text,29
label,20


In [14]:
# replacing the missing values with null string
news_data = news_data.fillna('')

In [16]:
# merging th eauthor name and news title
news_data['content'] = news_data['text']+' '+news_data['title']

In [17]:
# first 5 rows of the dataset
news_data.head()

,Unnamed: 0,title,text,label,content
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,No comment is expected from Barack Obama Membe...
1,1,,Did they post their votes for Hillary already?,1,Did they post their votes for Hillary already?
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,"Now, most of the demonstrators gathered last ..."
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,A dozen politically active pastors came here f...
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,"The RS-28 Sarmat missile, dubbed Satan 2, will..."


In [18]:
# separating feature and target
X = news_data.drop(columns='label', axis =1)
Y = news_data['label']

In [19]:
print(X)

      Unnamed: 0                                              title  \
0              0  LAW ENFORCEMENT ON HIGH ALERT Following Threat...   
1              1                                                      
2              2  UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...   
3              3  Bobby Jindal, raised Hindu, uses story of Chri...   
4              4  SATAN 2: Russia unvelis an image of its terrif...   
...          ...                                                ...   
18974      18954  Americans Are So Disconnected from Reality Tha...   
18975      18955  Michigan governor asks Obama for federal aid i...   
18976      18956  House will not reach healthcare deal before tw...   
18977      18957  Erdogan says Turkey must clear Syria's Afrin o...   
18978      18958   WATCH: 20 Years Ago, David Bowie Saw America’...   

                                                    text  \
0      No comment is expected from Barack Obama Membe...   
1         Did they post the

In [20]:
print(Y)

0        1
1        1
2        1
3        0
4        1
        ..
18974    1
18975    0
18976    0
18977    0
18978    1
Name: label, Length: 18979, dtype: object


Stemming:

Stemming is the process of reducing a word to its Root Word

In [21]:
port_stem = PorterStemmer()

In [22]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ',content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [23]:
news_data['content'] = news_data['content'].apply(stemming)

In [24]:
print(news_data['content'])

0        comment expect barack obama member fyf fukyofl...
1                                post vote hillari alreadi
2        demonstr gather last night exercis constitut p...
3        dozen polit activ pastor came privat dinner fr...
4        rs sarmat missil dub satan replac ss fli mile ...
                               ...                        
18974    recipi email idiot presstitut brainwash victim...
18975    detroit reuter michigan governor rick snyder a...
18976    washington reuter deep divis cut short republi...
18977    ankara reuter presid tayyip erdogan said frida...
18978    almost year ago david bowi releas music video ...
Name: content, Length: 18979, dtype: object


In [25]:
X = news_data['content'].values
Y = news_data['label'].values

In [26]:
print(X)

['comment expect barack obama member fyf fukyoflag blacklivesmatt movement call lynch hang white peopl cop encourag other radio show tuesday night turn tide kill white peopl cop send messag kill black peopl america one f yoflag organ call sunshin radio blog show host texa call sunshin f ing opinion radio show snapshot fyf lolatwhitefear twitter page p show urg support call fyf tonight continu dismantl illus white snapshot twitter radio call invit fyf radio show air p eastern standard time show caller clearli call lynch kill white peopl minut clip radio show heard provid breitbart texa someon would like refer hannib alreadi receiv death threat result interrupt fyf confer call unidentifi black man said mother f ker start f ing like us bunch ni er takin one us roll said caus alreadi roll gang anyway six seven black mother f cker see white person lynch ass let turn tabl conspir cop start lose peopl state emerg specul one two thing would happen big ass r war ni er go start backin alreadi ge

In [27]:
print(Y)

['1' '1' '1' ... '0' '0' '1']


In [28]:
Y.shape

(18979,)

In [2]:
# converting the textual data to feature vectors
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

NameError: name 'TfidfVectorizer' is not defined

In [1]:
print(X)

NameError: name 'X' is not defined